In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math as mt

In [2]:
peak_df = pd.DataFrame(columns=['date','peak (kW)'])
household_kWh_df = pd.DataFrame(columns=['name', 'usage (kWh)'])
now_month = 2

# Set Peak Data
year = 2020
months = [3,4,5,6,7,8,9,10,11,12,1,2]
peak_kW = [95,95,94,90,91,72,66,87,91,85,100,95]
household_kWh = [150, 180, 220, 210, 310, 300, 270, 190, 250, 260]

for idx,month in enumerate(months):
    if month == 1:
        year += 1
    peak_df = peak_df.append({
        "date": dt.datetime(
            year,
            month,
            1
        ),
        "peak (kW)": peak_kW[idx]
    }, ignore_index=True)
    
peak_df['peak (kW)'] = pd.to_numeric(peak_df['peak (kW)'])
    
# Set Household Data
for _ in range(0, 10):
    household_kWh_df = household_kWh_df.append({
        "name": str(_ + 1) + "01호",
        "usage (kWh)": household_kWh[_]
    }, ignore_index=True)
household_kWh_df['usage (kWh)'] = pd.to_numeric(
                household_kWh_df['usage (kWh)']
        )

peak_df
household_kWh_df

,name,usage (kWh)
0,101호,150
1,201호,180
2,301호,220
3,401호,210
4,501호,310
5,601호,300
6,701호,270
7,801호,190
8,901호,250
9,1001호,260


In [4]:
household_kWh_df

,name,usage (kWh)
0,101호,150
1,201호,180
2,301호,220
3,401호,210
4,501호,310
5,601호,300
6,701호,270
7,801호,190
8,901호,250
9,1001호,260


In [289]:
# 1~6, 9~12
low_pressure_fee = [
    [0,200,910,88.3],
    [201,400,1600, 182.9],
    [401,float('inf'),7300, 275.6]
]
# 7~8
low_pressure_fee_summer = [
    [0,300,910,88.3],
    [301,450,1600,182.9],
    [451,float('inf'),7300,275.6]
]
# 1~6, 9~12
high_pressure_fee = [
    [0,200,730,73.3],
    [201,400,1260,142.3],
    [400,float('inf'),6060,210.6]
]
# 7~8
high_pressure_fee_summer = [
    [0,300,730,73.3],
    [301,450,1260,142.3],
    [451,float('inf'),6060,210.6]
]

general_fee = {
    "저압": [6160, 100.7, 60.2, 87.3],
    "고압 A": [
        [7170, 110.9, 66.9, 98.6],
        [8230, 106.9, 62.6, 93.3]
    ],
    "고압 B": [
        [7170, 108.8, 65.8, 95.6],
        [8230, 103.5, 60.5, 90.3]
    ]
}
env_fee = 5.3
fuel_fee = 3
VAT = 0.1
FUND = 0.037

class Household:
    def __init__(self,
                name, kwh, contract, contract_name):
        self.name = name
        self.kwh = kwh
        self.contract = contract
        self.contract_name = contract_name
        
    def set_bill(self,
                public_fee):
        self.self_fee = self.elec_bill_vat_fund
        self.public_fee = public_fee
        self.bill = self.self_fee + self.public_fee
        
    @property
    def basic(self):
        fee = None
        for _ in self.contract:
            if _[0] <= self.kwh and _[1] >= self.kwh:
                fee = _[2]
                break
        return fee
    
    @property
    def elec_rate(self):
        kwh = self.kwh
        fee = 0
        for _ in self.contract:
            if kwh <= _[1]:
                fee += (kwh * _[3])
                break
            else:
                kwh -= _[1]
                fee += (_[1] * _[3])
        
        return mt.floor(fee)
        
    @property
    def guarantee(self):
        if self.kwh <= 200:
            if self.contract_name == "종합계약":
                return 4000
            elif self.contract_name == "단일계약":
                return 2500
        else:
            return 0
        
    @property
    def elec_bill(self):
        return self.basic + self.elec_rate\
            - self.guarantee + self.env - self.fuel
    
class Public:
    def __init__(self,
                kwh,
                charge_applied,
                contract):
        # 공용설비사용량
        self.kwh = kwh
        # 요금적용전력
        self.charge_applied = charge_applied
        self.contract = contract
        
    @property
    def basic(self):
        return mt.floor(self.charge_applied * self.contract[0])
    
    @property
    def elec_rate(self):
        return mt.floor(self.kwh * self.contract[1])
    
    @property
    def elec_bill(self):
        return self.basic + self.elec_rate + self.env - self.fuel
    
@property
def env(self):
    return mt.floor(self.kwh * env_fee)
@property
def fuel(self):
    return mt.floor(self.kwh * fuel_fee)
@property
def vat(self):
    return round(self.elec_bill * 0.1)
@property
def fund(self):
    return mt.floor(self.elec_bill * 0.037 * 0.1) * 10
@property
def elec_bill_vat_fund(self):
    return mt.floor((self.elec_bill + self.vat + self.fund) * 0.1) * 10

Household.env = env
Household.fuel = fuel
Household.vat = vat
Household.fund = fund
Household.elec_bill_vat_fund = elec_bill_vat_fund

Public.env = env
Public.fuel = fuel
Public.vat = vat
Public.fund = fund
Public.elec_bill_vat_fund = elec_bill_vat_fund
    
class ManagementOffice:
    def __init__(self, 
                 month, peaks, households, APT, # datas
                 contract,
                 general_fee_info=None):
        print("[관리사무소] Init.")
        
        self.peaks = peaks
        self.APT = APT
        
        self.contract = contract
        self.select_fee(month, contract, general_fee_info)
        
        if contract == "종합계약":
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee[0],
                        contract_name=contract
                    )
                )

            # 공용설비사용요금
            households_kwh = sum([_.kwh for _ in self.households])
            public_kwh = APT - households_kwh
            max_peak = peak_df['peak (kW)'].max()
            charge_applied = max_peak * (public_kwh / APT)
            self.public = Public(
                kwh=public_kwh,
                charge_applied=charge_applied,
                contract=self.fee[1]
            )

            # 가구별 청구서 셋팅
            public_fee = self.public.elec_bill_vat_fund / len(self.households)
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

            # 아파트 청구서 셋팅
            bill = 0
            for household in self.households:
                bill += household.bill
            self.bill = bill
        elif contract == "단일계약":
            # 전체를 하나의 가구 본 상태에서 계약대로 계산식 진행
            # 후에 * len(household) 를 통해 아파트 전체 요금 통지서 확보
            num_household = len(households)
            mean_kwh = APT / num_household

            apart = Household(
                name="아파트",
                kwh=mean_kwh,
                contract=self.fee,
                contract_name=contract
            )
            basic = apart.basic * num_household
            elec_rate = apart.elec_rate * num_household
            env = apart.env * num_household
            fuel = apart.fuel * num_household
            
            elec_bill = basic + elec_rate + env - fuel
            self.bill = mt.floor((elec_bill \
                + round(elec_bill * 0.1) \
                + mt.floor(elec_bill * 0.037 * 0.1) * 10) * 0.1) * 10
            
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee,
                        contract_name=contract
                    )
                )
                
            # 공공설비사용요금
            households_bill = sum(
                [_.elec_bill_vat_fund for _ in self.households]
            )
            self.public_bill = self.bill - households_bill
            
            
            # 가구별 청구서 셋팅
            public_fee = self.public_bill / len(self.households)
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

    # 요금제 셋팅 메서드
    def select_fee(self, month, contract, general_fee_info):
            print("[관리사무소] 계약 정보 셋팅")

            if contract == "종합계약":
                household_fee = None
                public_fee = None
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    household_fee = low_pressure_fee
                else:
                    household_fee = low_pressure_fee_summer

                if general_fee_info == None:
                    raise Exception("종합계약은 일반용 전력 정보를 포함해야 합니다.\n")

                tmp_general_fee = None
                if general_fee_info == "저압":
                    tmp_general_fee = general_fee[general_fee_info]
                elif len(general_fee_info) == 2:
                    tmp_general_fee = general_fee[general_fee_info[0]]
                    tmp_general_fee = tmp_general_fee[general_fee_info[1]]
                else: 
                    raise Exception("일반용 전력 설정이 올바르지 않습니다\n"\
                                + "저압, [고압 A, 0 or 1], [고압 B, 0 or 1]")

                if month >= 6 and month <= 8:
                    public_fee = [tmp_general_fee[0], tmp_general_fee[1]]
                elif (month >= 3 and month <= 5) or\
                      (month >= 9 and month <= 10):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[2]]
                elif (month >= 1 and month <= 2) or\
                      (month >= 11 and month <= 12):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[3]]
                self.fee = [household_fee, public_fee]
                
            elif contract == "단일계약":
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    self.fee = high_pressure_fee
                else:
                    self.fee = high_pressure_fee_summer

            print("[관리사무소] 계약 정보 셋팅 완료\n")
            print(self.fee)        
            print("")

In [290]:
# 종합계약 가구별 계산기 테스트
calc = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=household_kWh_df,
    APT=3000,
    contract="종합계약",
    general_fee_info=['고압 A', 1]
)

idx = 1
print("기본요금",calc.households[idx].basic)
print("전력량 요금",calc.households[idx].elec_rate)
print("기후환경요금",calc.households[idx].env)
print("연료비 조정액",calc.households[idx].fuel)
print("전기요금계",calc.households[idx].elec_bill)
print("부가가치세",calc.households[idx].vat)
print("전력산업기반기금",calc.households[idx].fund)
print("부가가치세, 전력산업기반기금 적용",calc.households[idx].elec_bill_vat_fund)

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[[0, 200, 910, 88.3], [201, 400, 1600, 182.9], [401, inf, 7300, 275.6]], [8230, 93.3]]

[관리사무소] 가구 객체화
기본요금 910
전력량 요금 15894
기후환경요금 954
연료비 조정액 540
전기요금계 13218
부가가치세 1322
전력산업기반기금 480
부가가치세, 전력산업기반기금 적용 15020


In [291]:
# 종합계약 공공설비사용요금 계산기 테스트
print("기본요금", calc.public.basic)
print("전력량 요금", calc.public.elec_rate)
print("기후환경요금", calc.public.env)
print("연료비 조정액", calc.public.fuel)
print("전기요금계", calc.public.elec_bill)
print("부가가치세", calc.public.vat)
print("전력산업기반기금", calc.public.fund)
print("부가가치세, 전력산업기반기금 적용", calc.public.elec_bill_vat_fund)

기본요금 181060
전력량 요금 61578
기후환경요금 3498
연료비 조정액 1980
전기요금계 244156
부가가치세 24416
전력산업기반기금 9030
부가가치세, 전력산업기반기금 적용 277600


In [292]:
# 종합계약 가구별 청구서 테스트
idx = 1
print("#########################")
print("가구명:", calc.households[idx].name)
print("청구금액:", calc.households[idx].bill)
print("#########################")

print("#######아파트 청구금액#######")
print("청구금액:", calc.bill)
print("#########################")

#########################
가구명: 201호
청구금액: 42780.0
#########################
#######아파트 청구금액#######
청구금액: 565920.0
#########################


In [293]:
# 단일계약 청구금액 테스트
calc_single = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=household_kWh_df,
    APT=3000,
    contract="단일계약"
)

print("#######아파트 청구금액#######")
print("청구금액:", calc_single.bill)
print("#########################")

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [400, inf, 6060, 210.6]]

[관리사무소] 가구 객체화
#######아파트 청구금액#######
청구금액: 350650
#########################


In [286]:
# 단일계약 가구별 계산기 테스트
calc_single = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=household_kWh_df,
    APT=3000,
    contract="단일계약"
)

idx = 1
print("기본요금",calc_single.households[idx].basic)
print("전력량 요금",calc_single.households[idx].elec_rate)
print("기후환경요금",calc_single.households[idx].env)
print("연료비 조정액",calc_single.households[idx].fuel)
print("전기요금계",calc_single.households[idx].elec_bill)
print("부가가치세",calc_single.households[idx].vat)
print("전력산업기반기금",calc_single.households[idx].fund)
print("부가가치세, 전력산업기반기금 적용",calc_single.households[idx].elec_bill_vat_fund)

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [400, inf, 6060, 210.6]]

[관리사무소] 가구 객체화
기본요금 730
전력량 요금 13194
기후환경요금 954
연료비 조정액 540
전기요금계 11838
부가가치세 1184
전력산업기반기금 430
부가가치세, 전력산업기반기금 적용 13450


In [294]:
# 단일계약 공공설비요금 테스트
calc_single = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=household_kWh_df,
    APT=3000,
    contract="단일계약"
)

print("####아파트공공설비 청구금액####")
print("청구금액:", calc_single.public_bill)
print("#########################")

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [400, inf, 6060, 210.6]]

[관리사무소] 가구 객체화
####아파트공공설비 청구금액####
청구금액: 112660
#########################


In [296]:
# 단일계약 가구별 청구서 테스트
idx = 2
print("#########################")
print("가구명:", calc_single.households[idx].name)
print("청구금액:", calc_single.households[idx].bill)
print("#########################")

print("#######아파트 청구금액#######")
print("청구금액:", calc_single.bill)
print("#########################")

#########################
가구명: 301호
청구금액: 33166.0
#########################
#######아파트 청구금액#######
청구금액: 350650
#########################


In [322]:
# 종합계약 표
comp_household = household_kWh_df.copy()

calc = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=comp_household,
    APT=3000,
    contract="종합계약",
    general_fee_info=['고압 A', 1]
)

households_fee = [
    _.elec_bill_vat_fund
    for _ in calc.households    
]
public_fee = [
    mt.floor(_.public_fee)
    for _ in calc.households    
]
households_bill = [
    mt.floor(_.bill)
    for _ in calc.households    
]

comp_household['전기요금계'] = households_fee
comp_household['공공설비사용요금'] = public_fee
comp_household['최종청구금액'] = households_bill

comp_household = comp_household.append({
    "name": "계",
    "usage (kWh)": sum([_.kwh for _ in calc.households]),
    "전기요금계": sum(households_fee),
    "공공설비사용요금": sum(public_fee),
    "최종청구금액": sum(households_fee) + sum(public_fee)
},ignore_index=True)

comp_household.index = comp_household['name']
comp_household.drop(['name'], axis=1,inplace=True)
comp_household = comp_household.T
comp_household

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[[0, 200, 910, 88.3], [201, 400, 1600, 182.9], [401, inf, 7300, 275.6]], [8230, 93.3]]

[관리사무소] 가구 객체화


name,101호,201호,301호,401호,501호,601호,701호,801호,901호,1001호,계
usage (kWh),150,180,220,210,310,300,270,190,250,260,2340
전기요금계,11930,15020,26620,24510,45580,43470,37150,16050,32940,35050,288320
공공설비사용요금,27760,27760,27760,27760,27760,27760,27760,27760,27760,27760,277600
최종청구금액,39690,42780,54380,52270,73340,71230,64910,43810,60700,62810,565920


In [323]:
# 단일계약 표
single_household = household_kWh_df.copy()

calc = ManagementOffice(
    month=2,
    peaks=peak_df, 
    households=single_household,
    APT=3000,
    contract="단일계약"
)

households_fee = [
    _.elec_bill_vat_fund
    for _ in calc.households    
]
public_fee = [
    mt.floor(_.public_fee)
    for _ in calc.households    
]
households_bill = [
    mt.floor(_.bill)
    for _ in calc.households    
]

single_household['전기요금계'] = households_fee
single_household['공공설비사용요금'] = public_fee
single_household['최종청구금액'] = households_bill

single_household = single_household.append({
    "name": "계",
    "usage (kWh)": sum([_.kwh for _ in calc.households]),
    "전기요금계": sum(households_fee),
    "공공설비사용요금": sum(public_fee),
    "최종청구금액": sum(households_fee) + sum(public_fee)
},ignore_index=True)

single_household.index = single_household['name']
single_household.drop(['name'], axis=1,inplace=True)
single_household = single_household.T
single_household

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [400, inf, 6060, 210.6]]

[관리사무소] 가구 객체화


name,101호,201호,301호,401호,501호,601호,701호,801호,901호,1001호,계
usage (kWh),150,180,220,210,310,300,270,190,250,260,2340
전기요금계,10870,13450,21900,20250,36700,35060,30130,14310,26840,28480,237990
공공설비사용요금,11266,11266,11266,11266,11266,11266,11266,11266,11266,11266,112660
최종청구금액,22136,24716,33166,31516,47966,46326,41396,25576,38106,39746,350650
